In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import chardet

%matplotlib inline

In [4]:
Camp_Pref = pd.read_csv('campaigns performance.csv',encoding='utf_8_sig',sep=',', header=2)

In [5]:
Camp_Pref.head(1)

,Week,Campaign,Campaign ID,Currency,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Search Impr. share,Search Lost IS (budget),Search Lost IS (rank)
0,"Week of Oct 26, 2020",Shopping – Smart IL1 #QA V1 תקין,11522694129,USD,0,2,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,100.00%,0.00%,0.00%


In [6]:
Camp_Pref.drop(['Currency'], axis =1, inplace = True)

In [7]:
# Updating the columns type for further calculations 

Camp_Pref['Clicks'] = Camp_Pref['Clicks'].str.replace(',' , '').astype('float64')
Camp_Pref['Impressions'] = Camp_Pref['Impressions'].str.replace(',' , '').astype('float64')
Camp_Pref['CTR'] = Camp_Pref['CTR'].astype('str').str.rstrip('%').astype('float64')
Camp_Pref['Avg. CPC'] = Camp_Pref['Avg. CPC'].astype('float64')
Camp_Pref['Cost'] = Camp_Pref['Cost'].str.replace(',' , '').astype('float64')
Camp_Pref['Impr. (Abs. Top) %'] = Camp_Pref['Impr. (Abs. Top) %'].astype('str').str.rstrip('%').astype('float64')
Camp_Pref['Impr. (Top) %'] = Camp_Pref['Impr. (Top) %'].astype('str').str.rstrip('%').astype('float64')
Camp_Pref['Conversions'] = Camp_Pref['Conversions'].astype('float64')
Camp_Pref['Conv. rate'] = Camp_Pref['Conv. rate'].astype('str').str.rstrip('%').astype('float64')
Camp_Pref['Cost / conv.'] = Camp_Pref['Cost / conv.'].str.replace(',' , '').astype('float64')

In [8]:
# >90 will change to 90% and <10 will be changed to 10%
# (This metrics shows how well did our add worked, but for high and low rated is doesnt give a good approximation)

Camp_Pref['Search Lost IS (rank)'] = Camp_Pref['Search Lost IS (rank)'].replace(' --', np.nan)
Camp_Pref['Search Lost IS (rank)'] = Camp_Pref['Search Lost IS (rank)'].replace('>90%', '90%')
Camp_Pref['Search Lost IS (rank)'] = Camp_Pref['Search Lost IS (rank)'].replace('<10 %','10%').astype('str').str.rstrip('%').astype('float64')

Camp_Pref['Search Impr. share'] = Camp_Pref['Search Impr. share'].replace(' --', np.nan)
Camp_Pref['Search Impr. share'] = Camp_Pref['Search Impr. share'].replace('>90%', '90%')
Camp_Pref['Search Impr. share'] = Camp_Pref['Search Impr. share'].replace('<10%','10%').astype('str').str.rstrip('%').astype('float64')

Camp_Pref['Search Lost IS (budget)'] = Camp_Pref['Search Lost IS (budget)'].replace(' --', np.nan)
Camp_Pref['Search Lost IS (budget)'] = Camp_Pref['Search Lost IS (budget)'].replace('>90%', '90%')
Camp_Pref['Search Lost IS (budget)'] = Camp_Pref['Search Lost IS (budget)'].replace('<10%','10%').astype('str').str.rstrip('%').astype('float64')

In [9]:
Camp_Pref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Week                     494 non-null    object 
 1   Campaign                 494 non-null    object 
 2   Campaign ID              494 non-null    int64  
 3   Clicks                   494 non-null    float64
 4   Impressions              494 non-null    float64
 5   CTR                      494 non-null    float64
 6   Avg. CPC                 494 non-null    float64
 7   Cost                     494 non-null    float64
 8   Impr. (Abs. Top) %       494 non-null    float64
 9   Impr. (Top) %            494 non-null    float64
 10  Conversions              494 non-null    float64
 11  View-through conv.       494 non-null    int64  
 12  Cost / conv.             494 non-null    float64
 13  Conv. rate               494 non-null    float64
 14  Search Impr. share       2

In [10]:
Camp_Pref['Week']= Camp_Pref['Week'].astype('str').str.lstrip(' Week of ')
Camp_Pref['Week'] = pd.to_datetime(Camp_Pref['Week']).dt.strftime('%Y-%m')

In [11]:
# Creating a pivot table with multi-index of Campaign's ID and the date
    
Preformance_pivot1 = pd.pivot_table(data = Camp_Pref, index= ['Campaign ID','Week'], 
                                  values = ['Clicks', 'Impressions','Cost','Conversions', 'View-through conv.'],
                                  aggfunc=np.sum)

In [12]:
Preformance_pivot1['total conv.'] = Preformance_pivot1['Conversions']+ Preformance_pivot1['View-through conv.']

In [13]:
# Creating a summary to each columns with sum function, please note in the next command line the summary line won't
# contain only sum but the folllowing calculations as well
Preformance_pivot1 = pd.concat([d.append(d.sum().rename((k, 'Summary'))) for k, d in Preformance_pivot1.groupby(level=0)])

In [14]:
Preformance_pivot1['CTR'] = (Preformance_pivot1['Clicks']/Preformance_pivot1['Impressions'])*100
Preformance_pivot1['Avg. CPC'] = Preformance_pivot1['Cost']/Preformance_pivot1['Clicks']
Preformance_pivot1['Cost / conv.'] = Preformance_pivot1['Cost']/Preformance_pivot1['total conv.']
Preformance_pivot1['Conv. rate'] = (Preformance_pivot1['total conv.']/Preformance_pivot1['Clicks'])*100

In [15]:
column_order = ['Cost', 'Impressions', 'Clicks', 'Conversions', 'View-through conv.', 'total conv.',
               'CTR',  'Conv. rate', 'Avg. CPC', 'Cost / conv.']
Preformance_pivot1 = Preformance_pivot1.reindex(column_order, axis=1)
total_sum = Preformance_pivot1.loc[(slice(None), 'Summary'), :]

In [16]:
for x in ['CTR','Conv. rate','total conv.','Avg. CPC', 'Cost / conv.']:
    print (x)
    print ("median - " + str(total_sum[x].median()) )
    print ("mean - " + str(total_sum[x].mean()) )
    print ("max - " + str(total_sum[x].max()) )
    print ('')
   

CTR
median - 0.9603787336416348
mean - 2.8731684854605812
max - 45.78675321553071

Conv. rate
median - 0.07329821417268612
mean - 4.024035585233939
max - 257.14285714285717

total conv.
median - 0.825
mean - 69.72568421052632
max - 3710.25

Avg. CPC
median - 0.7020564516129032
mean - inf
max - inf

Cost / conv.
median - 488.0633333333334
mean - inf
max - inf



In [17]:
# Calculating most profitable campaigns using total Conv. metric and cost/conv. # 

In [18]:
total_sum_top = total_sum[total_sum['total conv.']>=300]

total_sum_top = total_sum_top[total_sum['Cost / conv.']<=100]

total_sum_top = total_sum_top.sort_values(by=['total conv.'], ascending=False)
total_sum_top = total_sum_top[['Cost' ,'Conversions', 'View-through conv.', 'total conv.','CTR', 'Conv. rate', 'Avg. CPC','Cost / conv.']]
total_sum_top

<ipython-input-18-f439c0a59c7d>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_sum_top = total_sum_top[total_sum['Cost / conv.']<=100]


,,Cost,Conversions,View-through conv.,total conv.,CTR,Conv. rate,Avg. CPC,Cost / conv.
Campaign ID,Week,,,,,,,,
10419629499,Summary,13241.48,3710.25,0.0,3710.25,12.816543,7.610145,0.271598,3.568892
11508644810,Summary,19183.86,151.00,2131.0,2282.00,1.074741,2.589768,0.217711,8.406599
11268506295,Summary,28349.94,129.00,1494.0,1623.00,0.560533,1.099795,0.192108,17.467616
11661592991,Summary,16616.09,141.00,1083.0,1224.00,0.157082,5.569712,0.756102,13.575237
11604736493,Summary,5740.98,78.00,723.0,801.00,0.955157,3.810113,0.273081,7.167266


In [19]:
# New pivot table with Search impression and Search lost metrics

Preformance_pivot2 = pd.pivot_table(data = Camp_Pref, index= ['Campaign ID','Week'], 
                                  values = ['Search Impr. share', 'Search Lost IS (budget)', 'Search Lost IS (rank)'],
                                  aggfunc=np.average)
# creating a subtotal with agg-fun - average
Preformance_pivot2 = pd.concat([d.append(d.mean().rename((k, 'Average'))) for k, d in Preformance_pivot2.groupby(level=0)])
total_avg = Preformance_pivot2.loc[(slice(None), 'Average'), :]

In [20]:
print ('Search Impr. share avg - ' + str(total_avg['Search Impr. share'].mean()))
print ('Search Lost IS (rank) avg- ' + str(total_avg['Search Lost IS (rank)'].mean()))
print ('Search Lost IS (budget) avg- ' + str(total_avg['Search Lost IS (budget)'].mean()))

Search Impr. share avg - 44.65140003052503
Search Lost IS (rank) avg- 52.314513888888904
Search Lost IS (budget) avg- 3.2442539682539686


In [21]:
merged_top = total_sum_top.merge(total_avg, 'left', on =['Campaign ID'], left_index=True )
merged_top

,Cost,Conversions,View-through conv.,total conv.,CTR,Conv. rate,Avg. CPC,Cost / conv.,Search Impr. share,Search Lost IS (budget),Search Lost IS (rank)
Campaign ID,,,,,,,,,,,
10419629499,13241.48,3710.25,0.0,3710.25,12.816543,7.610145,0.271598,3.568892,25.775,0.855,73.37
11508644810,19183.86,151.00,2131.0,2282.00,1.074741,2.589768,0.217711,8.406599,NaN,NaN,NaN
11268506295,28349.94,129.00,1494.0,1623.00,0.560533,1.099795,0.192108,17.467616,NaN,NaN,NaN
11661592991,16616.09,141.00,1083.0,1224.00,0.157082,5.569712,0.756102,13.575237,NaN,NaN,NaN
11604736493,5740.98,78.00,723.0,801.00,0.955157,3.810113,0.273081,7.167266,NaN,NaN,NaN


In [22]:
Campaign_id = list(merged_top.index)
Campaign_id

[10419629499, 11508644810, 11268506295, 11661592991, 11604736493]

In [23]:
# Extracting the names and ID's of top profitable campaigns
Campaign_name_id = Camp_Pref[['Campaign', 'Campaign ID']].drop_duplicates()
Campaign_name_id = Campaign_name_id[Campaign_name_id['Campaign ID'].isin(Campaign_id)]
top_campaign = list(Campaign_name_id['Campaign'])
Campaign_name_id

,Campaign,Campaign ID
2,GM - REM1 - NA #QA V2,11268506295
49,Smart Global - TOP COUNTRY 1 #QA WL V1 מעקב צמוד,11661592991
102,Discovery - REM - Global #QA V1 RG1,11508644810
115,Global - Branded #QA,10419629499
117,Discovery - REM - Global #QA V1 RG2 תקין,11604736493
